# K-Nearest Neighbors (K-NN)

### 參考課程實作並在datasets_483_982_spam.csv的資料集中獲得90% 以上的 accuracy (testset)

## Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import codecs
import re

## Importing the dataset

In [8]:
dataset = pd.read_csv(r'datasets_483_982_spam.csv', encoding = 'latin-1')
dataset['is_spam'] = dataset['v1'].map({'ham': 0, 'spam': 1})
all_data = dataset[['v2', 'is_spam']].values

In [11]:
dataset.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,is_spam
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,0
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,1
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,0


In [14]:
all_data

array([['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
        0],
       ['Ok lar... Joking wif u oni...', 0],
       ["Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
        1],
       ...,
       ['Pity, * was in mood for that. So...any other suggestions?', 0],
       ["The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
        0],
       ['Rofl. Its true to its name', 0]], dtype=object)

### 取出訓練內文與標註

In [15]:
X = all_data[:,0]
Y = all_data[:,1].astype(np.uint8)

In [16]:
print('Training Data Examples : \n{}'.format(X[:5]))

Training Data Examples : 
['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'
 'Ok lar... Joking wif u oni...'
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"
 'U dun say so early hor... U c already then say...'
 "Nah I don't think he goes to usf, he lives around here though"]


In [17]:
print('Labeling Data Examples : \n{}'.format(Y[:5]))

Labeling Data Examples : 
[0 0 1 0 0]


### 文字預處理

In [18]:
from sklearn.metrics import confusion_matrix
from nltk.corpus import stopwords

import nltk

nltk.download('stopwords')

# Lemmatize with POS Tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 

## 創建Lemmatizer
lemmatizer = WordNetLemmatizer() 

def get_wordnet_pos(word):
    """將pos_tag結果mapping到lemmatizer中pos的格式"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


def clean_content(X):
    stop_words = set(stopwords.words('english'))
    
    X_output = []
    for x in X:
        clean = re.sub('[^a-zA-Z]',' ', x).lower()
        tokens = nltk.word_tokenize(clean) 
        clean_tokens = []
        for token in tokens:
            if token not in stop_words:
                word = lemmatizer.lemmatize(token, get_wordnet_pos(token))
                clean_tokens.append(word)
        X_output.append(' '.join(clean_tokens))
    
    return X_output               

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
X = clean_content(X)

### Bag of words

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
#max_features是要建造幾個column，會按造字出現的高低去篩選 
cv=CountVectorizer(max_features = 1000)
X=cv.fit_transform(X).toarray()

In [21]:
X.shape

(5572, 1000)

## Splitting the dataset into the Training set and Test set

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

## Training the K-NN model on the Training set

In [23]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2, n_jobs=-1)
classifier.fit(X_train, y_train)

KNeighborsClassifier(n_jobs=-1)

## Predicting a new result

In [24]:
print('Trainset Accuracy: {}'.format(classifier.score(X_train, y_train)))

Trainset Accuracy: 0.943459726273278


In [25]:
print('Testset Accuracy: {}'.format(classifier.score(X_test, y_test)))

Testset Accuracy: 0.9174887892376682


## Predicting the Test set results

In [26]:
y_pred = classifier.predict(X_test)

## Making the Confusion Matrix

In [27]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[949   0]
 [ 92  74]]


0.9174887892376682

In [28]:
y_test = pd.Series(y_test).map({0: 'ham', 1: 'spam'})
y_pred = pd.Series(y_pred).map({0: 'ham', 1: 'spam'})
cm = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
print(cm)

Predicted   ham  spam   All
Actual                     
ham         949     0   949
spam         92    74   166
All        1041    74  1115
